Reuters Data Set
Working through Text Mining and Visualization: Case Studies using Open-Source Tools
Chapman & Hall/CRC Data Mining and Knowledge Discovery    
http://www.nltk.org/book/ch02.html help is available in the nltk book section 1.4
http://enricbaltasar.com/python-summary-methods-lists/ help with Python lists
http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
Pandas dataframe guide

Investigating word popularity in the articles

In [168]:
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import StanfordTokenizer
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
from nltk.corpus import reuters
import os
from nltk.parse import stanford
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import RegexpStemmer
st = RegexpStemmer('ing$|s$|e$|able$', min=4)
from nltk.stem.snowball import EnglishStemmer
snow = EnglishStemmer()
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from collections import Counter
import numpy as np
import pandas as pd
#os.environ['STANFORD_PARSER'] = '/Users/David/postagger/stanford-parser-full-2014-08-27/stanford-parser.jar'
#os.environ['STANFORD_MODELS'] = '/Users/David/postagger/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models.jar'
#parser = stanford.StanfordParser(model_path="/Users/David/postagger/stanford-parser-full-2014-08-27/stanford-parser-3.4.1-models/edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
cachedStopWords = stopwords.words("english")
#cachedStopWords.append('said')
 
def tokenize(text):
    min_length = 3
    #tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
    words =  map(lambda word: word.lower(), WordPunctTokenizer().tokenize(text))
    #words = list(WhitespaceTokenizer().span_tokenize(text))
    #words = map(lambda word: word.lower(), word_tokenize(text));    
    #words = wordpunct_tokenize(text)
    #word_tokenize = Penn Treebank#
    #words = map(lambda word: word.lower(), tokenizer.tokenize(text));
    #words = list(parser.raw_parse(text,verbose=False))
    #words = map(lambda word: word.lower(), text.split())
    words = [word for word in words
             if word not in cachedStopWords]
    #tokens =(list(map(lambda token: PorterStemmer().stem(token),words)));
    #tokens =(list(map(lambda token: LancasterStemmer().stem(token),words)));
    #tokens =(list(map(lambda token: snow.stem(token),words)));
    #tokens =(list(map(lambda token: st.stem(token),words)));
    tokens =(list(map(lambda token: wnl.lemmatize(token),words)));
    #tokens=words
    p = re.compile('[a-zA-Z]+');
    filter_words = list(filter(lambda token: p.match(token) and len(token)>=min_length, tokens));
    return filter_words


In [169]:
#build a data frame to feed the vectorizer
category_docs = reuters.fileids();
names = ['file_id', 'category','text']
file_id = 'x'
category = 'x'
text = 'x'

df = pd.DataFrame([[file_id,category,text]],columns=names)

for document_id in category_docs:
    text = reuters.raw(document_id)
    category =(reuters.categories(document_id))
    file_id = (document_id)
    temdf = pd.DataFrame([[file_id,category,text]],columns=names)
    if (len(category) == 1):
        df = df.append(temdf, ignore_index=True)
    
df['part'] = df['file_id'].apply(lambda x: x[0:5])


In [170]:
df.shape


(9161, 4)

In [171]:
df.tail(5)

,file_id,category,text,part
9156,training/9988,[interest],FED SETS TWO BILLION DLR CUSTOMER REPURCHASE\n...,train
9157,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...,train
9158,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...,train
9159,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...,train
9160,training/9995,[earn],&lt;A.H.A. AUTOMOTIVE TECHNOLOGIES CORP> YEAR ...,train


In [172]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#Values for document frequencies in percent.
minp=0.001
maxp=0.95
vec6_count = CountVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')
vec6_idf =   TfidfVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=10)
#hv.transform(corpus)
vec=vec6_count

#2# Specify the model
from sklearn.naive_bayes import MultinomialNB
#clf = MultinomialNB()
from sklearn import neighbors
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn import svm
#clf = svm.SVC()
#n_neighbors = 15
#clf = neighbors.KNeighborsClassifier(n_neighbors)
#clf = neighbors.RadiusNeighborsClassifier(n_neighbors)
from sklearn.neural_network import MLPClassifier
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#3# Using pipelineing, the data transformations can chained together 
#and the last object is the estimator or model.

from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")
from sklearn import pipeline
pipe = pipeline.Pipeline([('vectorizer', vec),
                          ('clf', clf)])

In [173]:
X=df['text']
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['category'])
y=le.transform(df['category'])

In [174]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.4,random_state=79)

In [175]:
#6# Train the model using 5 fold cross validation and calculate the scores. 
train_scores = cross_validation.cross_val_score(pipe, X_train, y_train, cv=5)
#Fit the model
pipe=pipe.fit(X_train, y_train)
#Score the model on unseen test data  data
test_score=pipe.score(X_test,y_test)

#7# Print out the accuracy scores and 
print("Training Accuracy: %0.3f (+/- %0.3f). Test Score: (%0.3f)" 
      % (train_scores.mean(), train_scores.std(),test_score))




Training Accuracy: 0.924 (+/- 0.011). Test Score: (0.924)


In [176]:
y_predicted = pipe.predict(df['text'])
pred_labels = le.inverse_transform(y_predicted)
act_labels = le.inverse_transform(y_test)

In [177]:
df['predicted'] = pred_labels

In [178]:
df['score'] = (df['predicted'] == df['category'])
df['hash'] = 1

In [179]:
temdf = df.drop(['text'], axis=1, level=None, inplace=False, errors='raise')
temdf.head(5)
temdf = temdf[temdf['file_id'] != 'x']
temdf['category'] = temdf['category'].apply(lambda x: '' + str(x[0]))

In [180]:
table = pd.pivot_table(temdf, values='hash', index='category', columns='score', aggfunc=np.sum, margins=True, fill_value=0)
table = table.sort('All', ascending=False)
table.head(10)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


score,False,True,All
category,,,
All,295.0,8865.0,9160.0
earn,30.0,3893.0,3923.0
acq,27.0,2265.0,2292.0
crude,22.0,352.0,374.0
trade,22.0,304.0,326.0
money-fx,36.0,273.0,309.0
interest,17.0,255.0,272.0
money-supply,11.0,140.0,151.0
ship,11.0,133.0,144.0


In [181]:
table.iloc[:,1] == table.iloc[:,2]
print(table[table.iloc[:,1] == table.iloc[:,2]])

score        False  True   All
category                      
copper         0.0  44.0  44.0
cotton         0.0  24.0  24.0
orange         0.0  22.0  22.0
housing        0.0  17.0  17.0
lei            0.0  14.0  14.0
carcass        0.0  11.0  11.0
instal-debt    0.0   6.0   6.0
tea            0.0   5.0   5.0
cpu            0.0   4.0   4.0
jet            0.0   3.0   3.0
naphtha        0.0   1.0   1.0
l-cattle       0.0   1.0   1.0
dmk            0.0   1.0   1.0
hog            0.0   1.0   1.0
rice           0.0   1.0   1.0
rand           0.0   1.0   1.0
propane        0.0   1.0   1.0


In [182]:
print(table[table.iloc[:,0] == table.iloc[:,2]])

score     False  True  All
category                  
coconut     1.0   0.0  1.0
nzdlr       1.0   0.0  1.0


In [167]:
table.shape

(23, 3)

In [113]:
temdf.head(10)

,file_id,category,part,predicted,score,hash
1,test/14826,[u'trade'],test/,[trade],True,1
2,test/14828,[u'grain'],test/,[grain],True,1
3,test/14839,[u'ship'],test/,[ship],True,1
4,test/14842,[u'gold'],test/,[gold],True,1
5,test/14843,[u'acq'],test/,[acq],True,1
6,test/14844,[u'tin'],test/,[naphtha],False,1
7,test/14854,[u'ipi'],test/,[ipi],True,1
8,test/14859,[u'earn'],test/,[earn],True,1
9,test/14860,[u'earn'],test/,[earn],True,1
10,test/14865,[u'acq'],test/,[gold],False,1


In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import neighbors
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn import svm
n_neighbors = 15
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2")
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = MultinomialNB()
clf5 = svm.SVC()
clf6 = neighbors.KNeighborsClassifier(n_neighbors)
clf7 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('mnb', clf4), ('svm', clf5),('knn', clf6), 
                                    ('mlp', clf7), ('sgd', clf)], voting='hard')
vec6_count = CountVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')
vec6_idf =   TfidfVectorizer(tokenizer=tokenize,min_df=minp,max_df=maxp,ngram_range=(1,3),analyzer='word')

for vec, vec_lab in zip([vec6_count, vec6_idf],['Count vec','tfidf vec']):
    for clf, label in zip([clf, clf1, clf2, clf4, clf5, clf6, clf7, eclf], 
                          ['Stocastic Gradient','Logistic Regression', 'Random Forest', 'naive Bayes', 
                           'svm','knn','nn mlp', 'ensemble']):
        pipe = pipeline.Pipeline([('vectorizer', vec),('clf', clf)])
        train_scores = cross_validation.cross_val_score(pipe, X_train, y_train, cv=5)
        pipe=pipe.fit(X_train, y_train)
        test_score=pipe.score(X_test,y_test)
        print("Vec:  [%s] -> Accuracy: %0.2f (+/- %0.2f) Test: %0.2f  Model: [%s]" 
              % (vec_lab, train_scores.mean(), train_scores.std(), test_score, label))

Vec:  [Count vec] -> Accuracy: 0.92 (+/- 0.01) Test: 0.92  Model: [Stocastic Gradient]
Vec:  [Count vec] -> Accuracy: 0.94 (+/- 0.01) Test: 0.94  Model: [Logistic Regression]
Vec:  [Count vec] -> Accuracy: 0.84 (+/- 0.02) Test: 0.83  Model: [Random Forest]
Vec:  [Count vec] -> Accuracy: 0.88 (+/- 0.02) Test: 0.88  Model: [naive Bayes]
Vec:  [Count vec] -> Accuracy: 0.64 (+/- 0.01) Test: 0.66  Model: [svm]
Vec:  [Count vec] -> Accuracy: 0.70 (+/- 0.02) Test: 0.73  Model: [knn]
Vec:  [Count vec] -> Accuracy: 0.75 (+/- 0.04) Test: 0.73  Model: [nn mlp]
Vec:  [Count vec] -> Accuracy: 0.91 (+/- 0.01) Test: 0.91  Model: [ensemble]
Vec:  [tfidf vec] -> Accuracy: 0.87 (+/- 0.02) Test: 0.88  Model: [Stocastic Gradient]
Vec:  [tfidf vec] -> Accuracy: 0.87 (+/- 0.02) Test: 0.88  Model: [Logistic Regression]
Vec:  [tfidf vec] -> Accuracy: 0.84 (+/- 0.02) Test: 0.84  Model: [Random Forest]
Vec:  [tfidf vec] -> Accuracy: 0.77 (+/- 0.02) Test: 0.78  Model: [naive Bayes]
Vec:  [tfidf vec] -> Accuracy: